In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import math
from itertools import combinations
from sklearn.metrics import accuracy_score
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


C:\Users\82104\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# load data
train = pd.read_csv("./train.csv", index_col=0)
test = pd.read_csv("./test.csv", index_col=0)

In [3]:
train = train.dropna()
train = train.drop(train[train['dered_g']==-9999].index)
train = train.drop(train[train['dered_i']==-9999].index)
train = train.drop(train[train['dered_z']==-9999].index).reset_index(drop=True)

In [4]:
#유용한 값들 추가
train['u-g'] = train['u'] - train['g']
train['g-r'] = train['g'] - train['r']
train['r-i'] = train['r'] - train['i']
train['i-z'] = train['i'] - train['z']
train['dered_u-g'] = train['dered_u'] - train['dered_g']
train['dered_g-r'] = train['dered_g'] - train['dered_r']
train['dered_r-i'] = train['dered_r'] - train['dered_i']
train['dered_i-z'] = train['dered_i'] - train['dered_z']

test['u-g'] = test['u'] - test['g']
test['g-r'] = test['g'] - test['r']
test['r-i'] = test['r'] - test['i']
test['i-z'] = test['i'] - test['z']
test['dered_u-g'] = test['dered_u'] - test['dered_g']
test['dered_g-r'] = test['dered_g'] - test['dered_r']
test['dered_r-i'] = test['dered_r'] - test['dered_i']
test['dered_i-z'] = test['dered_i'] - test['dered_z']

In [5]:
airmass = train[['airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z']]
train["airmass"] = airmass.median(axis = 1)

In [6]:
train['nO-nD'] = train['nObserve'] - train['nDetect']
test['nO-nD'] = test['nObserve'] - test['nDetect']

In [7]:
train = train.drop(['nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z'], axis=1)

In [8]:
oneandtwo = train[train['class']!=0]

In [9]:
# my_palette = sns.color_palette("bright", 2)
# for a, b in combinations(['redshift', 'dered_u-g', 'dered_g-r', 'dered_r-i', 'dered_i-z', 'dered_u', 'airmass', 'nO-nD'], 2):
#     sns.scatterplot(x=a,
#                     y=b,
#                     hue='class',
#                     data=oneandtwo,
#                     palette=my_palette)
#     plt.show()

# sns.scatterplot(x='u-g',
#                 y='g-r',
#                 hue='class',
#                 data=oneandtwo,
#                 palette=my_palette)
# plt.show()

In [10]:
# train = train.drop(['nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z'], axis=1)
# test = test.drop(['nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z'], axis=1)

In [11]:
# # split data into X and y
# Y = train['class']
# del train['class']
# X = train

In [12]:
# # scale features
# scaler = MinMaxScaler()
# scaler.fit(X) # fit scaler to training data only
# scaled = pd.DataFrame(scaler.transform(X), columns=X.columns)
# check = pd.concat([scaled, Y], axis=1)

In [225]:
split1 = train[(train['dered_u-g'] >= 0.5) & (train['dered_u-g'] <= 1.1)]
split2 = train[train['dered_u-g'] <= 0.5]
split3 = train[train['dered_u-g'] >= 1.1]
# split1 = train[(train['dered_g-r'] >= 0) & (train['dered_g-r'] <= 0.53)]
# split2 = train[train['dered_g-r'] <= 0]
# split3 = train[train['dered_g-r'] >= 0.53]

In [226]:
split1['class'].value_counts()

1    31660
0    31642
2    26994
Name: class, dtype: int64

In [231]:
split1[split1['class']==1].describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,...,u-g,g-r,r-i,i-z,dered_u-g,dered_g-r,dered_r-i,dered_i-z,airmass,nO-nD
count,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,...,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000,31660.000000
mean,19.182432,18.258743,17.846300,17.556343,17.444051,0.104424,19.021438,18.133254,17.759528,17.493459,...,0.923689,0.412442,0.289957,0.112292,0.888184,0.373726,0.266069,0.096749,1.176930,0.249337
std,1.098126,1.119704,1.073361,1.109990,1.113740,0.074699,1.063138,1.095517,1.057690,1.062133,...,0.154367,0.227410,0.351756,0.398710,0.142061,0.222061,0.250894,0.288649,0.114265,1.148498
min,12.754494,11.822671,11.275360,-23.902399,8.925076,-0.000521,11.784170,11.016460,11.196000,10.911250,...,-2.983005,-8.785322,-30.490405,-37.435316,0.500180,-7.195340,-11.014240,-10.653880,1.002026,0.000000
25%,18.562089,17.610602,17.252483,16.960841,16.838915,0.050552,18.431955,17.507795,17.179720,16.908900,...,0.829141,0.308204,0.239722,0.063110,0.797877,0.275180,0.218820,0.049268,1.095351,0.000000
50%,19.089840,18.121577,17.721254,17.419295,17.313406,0.085033,18.951290,18.012330,17.649565,17.365690,...,0.947314,0.404290,0.306231,0.123540,0.914625,0.367470,0.284455,0.107370,1.180775,0.000000
75%,19.832916,18.969637,18.539103,18.221304,18.115937,0.138942,19.654100,18.830503,18.448085,18.152750,...,1.036164,0.512335,0.361121,0.176259,1.000420,0.472350,0.338782,0.157263,1.224668,0.000000
max,25.375485,24.238345,27.329625,46.572081,40.799011,0.551696,24.135940,23.524130,24.842680,27.625480,...,2.071612,2.798092,37.661133,8.786659,1.099990,3.567480,7.555860,10.929480,2.043130,34.000000


In [227]:
split2['class'].value_counts()

0    11597
1     1967
2      838
Name: class, dtype: int64

In [228]:
split3['class'].value_counts()

2    129384
0     76757
1      9157
Name: class, dtype: int64

In [16]:
split2 = split2[split2['class']!=1]
split = pd.concat([split1, split2])

In [17]:
split

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,...,u-g,g-r,r-i,i-z,dered_u-g,dered_g-r,dered_r-i,dered_i-z,airmass,nO-nD
1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,...,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,1.253254,0
2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,...,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,1.022499,0
9,20.686012,19.725662,19.388249,19.273578,19.259566,0.000343,20.61129,19.62088,19.32146,19.23181,...,0.960350,0.337413,0.114672,0.014012,0.99041,0.29942,0.08965,-0.05361,1.252748,0
10,19.041937,18.016629,17.672307,17.387389,17.229920,0.084437,18.89577,17.89833,17.59101,17.32773,...,1.025308,0.344322,0.284918,0.157469,0.99744,0.30732,0.26328,0.13734,1.142021,0
14,17.195800,16.133867,15.985079,15.987387,16.058278,-0.000431,17.15679,16.09979,15.96315,15.97266,...,1.061933,0.148788,-0.002308,-0.070891,1.05700,0.13664,-0.00951,-0.07905,1.194503,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319984,20.219232,18.242606,17.113460,16.444441,16.018847,-0.000319,19.93937,18.04103,16.97402,16.34054,...,1.976626,1.129146,0.669019,0.425594,1.89834,1.06701,0.63348,0.40086,1.288622,0
319986,19.322709,18.206920,17.763580,17.473582,17.314952,0.080070,19.27375,18.15245,17.72750,17.44869,...,1.115789,0.443340,0.289997,0.158631,1.12130,0.42495,0.27881,0.14030,1.027210,0
319988,19.472131,17.950884,17.178280,16.778657,16.476769,0.091239,19.33696,17.85176,17.10959,16.72726,...,1.521247,0.772604,0.399623,0.301888,1.48520,0.74217,0.38233,0.29029,1.116221,0
319989,18.894505,17.502664,16.947927,16.590124,16.385907,0.075422,18.67523,17.36901,16.85061,16.51367,...,1.391841,0.554737,0.357803,0.204217,1.30622,0.51840,0.33694,0.20078,1.203462,0


In [18]:
def distribution(sdss_df, axes, feature, row):
    '''Plot the distribution of a space object w.r.t. a given feature.'''
    labels = np.unique(sdss_df['class'])
    colors = ['skyblue', 'gold', 'red']
    for i in range(len(labels)):
        label = labels[i]
        ax = sns.distplot(sdss_df.loc[sdss_df['class']==label, feature], 
                          kde=False, bins=30, ax=axes[row, i], color=colors[i])
        ax.set_title(label)
        if (i == 0):
            ax.set(ylabel='Count')

def main():
    # define lists of relevant features
    nonugriv = ['dered_u-g', 'dered_g-r', 'dered_r-i', 'dered_i-z']
    ugriv = ['r_u-g', 'r_u-g', 'r_u-g', 'r_u-g']
    airmass = ['u-g+g-r', 'g-r+r-i', 'r-i+i-z']

    # plot the distribution of non-geo and non-ugriv features
#     length = len(nonugriv)
#     fig, axes = plt.subplots(nrows=length, ncols=3, figsize=(12, 14))
#     plt.subplots_adjust(wspace=.4, hspace=.4)
#     for row in range(length):
#         feat = nonugriv[row]
#         distribution(train, axes, feat, row)
#     plt.show()
        
#     # plot the distribution of ugriv features
#     length = len(ugriv)
#     fig, axes = plt.subplots(nrows=length, ncols=3, figsize=(12, 15))
#     plt.subplots_adjust(wspace=.4, hspace=.4)
#     for row in range(length):
#         feat = ugriv[row]
#         distribution(train, axes, feat, row)
#     plt.show()
    
#     # plot the distribution of ugriv features
#     length = len(airmass)
#     fig, axes = plt.subplots(nrows=length, ncols=3, figsize=(12, 15))
#     plt.subplots_adjust(wspace=.4, hspace=.4)
#     for row in range(length):
#         feat = airmass[row]
#         distribution(check, axes, feat, row)
#     plt.show()
    
main()

In [19]:
# for n in test.columns:
#     plt.figure(figsize=(13,3))
#     sns.kdeplot(test[f'{n}'],legend=True)